In [1]:
import numpy as np 
from Option_lib import * 
from Stochastic_processes import GBM, Heston93, Bates, VarianceGamma
from time import time
from datetime import datetime

### Pricing with the Geometric Brownian Motion

$$
\begin{split}
   &  dS_t = rS_t dt + \sigma S_t dW_t^Q \\
   & S_T = S_t\exp\left\{\left(r -\frac{\sigma^2}{2}\right)(T-t) + \sigma(W_T^Q-W_t^Q)\right\}
\end{split}
$$

Let's comapare the pricing functions (Monte-Carlo, Fourier and analytic Black-Scholes-Merton) for the European call options 

In [2]:
# Parameters  

np.random.seed(1)

r = 0.0
sigma = 0.2

T = 1 # Option maturity 
n_steps = 252 # steps = number of trading days

S0 = 90
K = 100

# creating the object

gbm_model = GBM(mu=r, sigma=sigma)

In [3]:
# Martingale check for the GBM

trj = gbm_model.simulate(S0, T, 10, 20)
print(np.mean(trj, axis=1))

[90.         89.99966063 89.99898587 89.99964566 90.00046348 90.00093573
 90.0035642  90.0058688  90.00414928 90.00408745 90.00122141]


In [4]:
# Pricing European call
eurocall_BS = gbm_model.BS_call_price(S0, K, T)
eurocall_damp = gbm_model.Fourier_call_pricing(S0, K, T) # Fourier pricing, damping method 
eurocall_cos = gbm_model.cos_pricing(S0, K, T, 'call') 
eurocall_mc = EuropeanOption(gbm_model.simulate(S0, T, n_steps, 20)[-1], K, T, r, 1)

print('European call\n')
print(f"Exact formula    : {eurocall_BS:>10.6f}")
print(f"Damping formula  : {eurocall_damp:>10.6f}")
print(f"COS formula      : {eurocall_cos:>10.6f}")
print(f"Monte-Carlo      : {eurocall_mc:>10.6f}")

European call

Exact formula    :   3.589108
Damping formula  :   3.589108
COS formula      :   3.589108
Monte-Carlo      :   3.586372


In [5]:
# Pricing Bermudan call option
n = 17
exercise_date = [0, 1/3, 2/3, 1] # exercise dates

St = gbm_model.simulate(S0, T, 3, n)
                   
bermudan_call_ls = AmericanOption(St, K, r, T, exercise_date, 1, 3)
bermudan_put_ls = AmericanOption(St, K, r, T, exercise_date, -1, 3)

print(f"Bermudan call (LS)      : {bermudan_call_ls:>10.6f}")
print(f"Bermudan put  (LS)      : {bermudan_put_ls:>10.6f}")

Bermudan call (LS)      :   3.573501
Bermudan put  (LS)      :  13.578266


### Pricing with the Heston model 

$$
\begin{split}
   &  dS_t = rS_t dt + \sqrt{\nu_t} S_t dW_{1,t}^Q \\
   & S_T = S_t\exp\left\{\left(r(T-t) -\int_t^{T}\frac{\nu_s}{2}ds\right) + \int_t^T\sqrt{\nu_t}dW_s^Q\right\} \\
   & d\nu_t = k(\theta -\nu_t)dt + \sqrt{\nu_t}dW_{2,t}^Q \\
   & E[dW_{1,t}^QdW_{2,t}^Q] = \rho dt 
\end{split}
$$

Let's comapare the pricing functions (Monte-Carlo and Fourier methods) for the European call options 

In [6]:
k = 1.5;             # speed of mean reversion of the vol process
theta = 0.04;        # long run memory of the vol process
nu0 = 0.04;          # initial value of the vol process
sigma = .2;          # diffusion parameter of the vol process
rho = -0.1;          # correlation of the Brownians (leverage effect)

N = 20

heston93_model = Heston93(r, k, theta, sigma, rho)

In [7]:
start = time()
Xt, nut = heston93_model.simulate(S0, nu0, T, 10, N, .5, .5, 'andersen');
end = time()
print(f'Elapsed time: {end-start:>10.6f}\n')
print(np.mean(Xt, axis=1))

Elapsed time:   7.147078

[90.         89.9916304  89.98447911 89.98774845 89.98528824 89.9884951
 89.98011452 89.98638063 89.9791883  89.98246405 89.98614645]


In [8]:
# Call pricing
start = time()
mc_heston93_call =  EuropeanOption(heston93_model.simulate(S0, nu0, T, n_steps, 16)[0][-1], K, T, r, 1)
fourier_heston93_call = heston93_model.Fourier_call_pricing(K, T, S0, nu0)
cos_heston93_call = heston93_model.cos_pricing(S0, nu0, K, T, 'call')
end = time()
print(f'Elapsed time: {end-start:>10.6f}\n')

print(f"Damping formula  : {fourier_heston93_call:>10.6f}")
print(f"COS formula      : {cos_heston93_call:>10.6f}")
print(f"Monte-Carlo      : {mc_heston93_call:>10.6f}")

Elapsed time:   3.033953

Damping formula  :   3.453119
COS formula      :   3.453177
Monte-Carlo      :   3.550271


In [9]:
# Pricing Bermudan call option
n = 17
exercise_date = [0, 1/3, 2/3, 1] # exercise dates

St, nut = heston93_model.simulate(S0, nu0, T, n_steps, n)
Xt = select_exercise_prices(St, exercise_date, len(St))

bermudan_call_ls = AmericanOption(Xt, K, r, T, exercise_date, 1, 3)
bermudan_put_ls = AmericanOption(Xt, K, r, T, exercise_date, -1, 3)

print(f"Bermudan call (LS)      : {bermudan_call_ls:>10.6f}")
print(f"Bermudan put  (LS)      : {bermudan_put_ls:>10.6f}")

Bermudan call (LS)      :   3.496331
Bermudan put  (LS)      :  13.475840


In [10]:
# Let's see how to do a calibration (on prices)

maturities = np.array([21/252, 42/252, 63/252, 84/252, 126/252])
strikes = np.arange(.9, 1.125, step=.025, ) * K

In [11]:
today = datetime.today()
data = np.empty((len(maturities), len(strikes)))
mkt_data = []
Options = Time_series_derivatives()

i=0
j=0 

for T in maturities:
    for K in strikes:
        cos_call_price = heston93_model.cos_pricing(S0, nu0, K, T, 'call', 50)
        data[i,j] = cos_call_price
        
        mkt_data.append([cos_call_price, S0, K, T, 'C'])
        options = VanillaOption(market_price=cos_call_price,
                                strike_price=K,
                                time_to_maturity=T,
                                typology='C',
                                style='E',
                                trading_date=today,
                                underlying_price=S0)
        
        Options.add_contract(options, today)
        
        j += 1 
    j=0 
    i+=1



In [15]:
# Let's perfom the calibration

params = np.array([k, theta, sigma, 0, nu0]) * 1.5

fitted_heston = Heston93(r)

start = time()
res = fitted_heston.calibrate(params, Options)
end = time()
print(f'Elapsed time: {end-start:>10.6f}')

Elapsed time: 251.351223


In [16]:
res.params

,params,gradient
k,2.252988,0.000043
theta,0.039705,-0.000153
sigma,0.218240,0.000011
rho,-0.097446,-0.000001
nu0,0.040099,-0.001314


In [17]:
res.fun

2.231811970854637e-05